In [1]:
%pylab inline
import findspark
findspark.init()
import pyspark

Populating the interactive namespace from numpy and matplotlib


In [2]:
sc = pyspark.SparkContext()

In [3]:
# load library 
import pandas as pd, numpy as np
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
import datetime
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [4]:
sqlCtx = SQLContext(sc)

# NYC Taxi data

In [5]:
# Read data
df = sqlCtx.read.format('com.databricks.spark.csv').options(header='true').load('train.csv')

In [6]:
df.dtypes

[('id', 'string'),
 ('vendor_id', 'string'),
 ('pickup_datetime', 'string'),
 ('dropoff_datetime', 'string'),
 ('passenger_count', 'string'),
 ('pickup_longitude', 'string'),
 ('pickup_latitude', 'string'),
 ('dropoff_longitude', 'string'),
 ('dropoff_latitude', 'string'),
 ('store_and_fwd_flag', 'string'),
 ('trip_duration', 'string')]

In [7]:
df = df.withColumn("pickup_longitude", df["pickup_longitude"].cast('double'))
df = df.withColumn("pickup_latitude", df["pickup_latitude"].cast('double'))
df = df.withColumn("dropoff_longitude", df["dropoff_longitude"].cast('double'))
df = df.withColumn("dropoff_latitude", df["dropoff_latitude"].cast('double'))

In [8]:
df.show(1)

+---------+---------+-------------------+-------------------+---------------+-----------------+-----------------+------------------+------------------+------------------+-------------+
|       id|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count| pickup_longitude|  pickup_latitude| dropoff_longitude|  dropoff_latitude|store_and_fwd_flag|trip_duration|
+---------+---------+-------------------+-------------------+---------------+-----------------+-----------------+------------------+------------------+------------------+-------------+
|id2875421|        2|2016-03-14 17:24:55|2016-03-14 17:32:30|              1|-73.9821548461914|40.76793670654297|-73.96463012695312|40.765602111816406|                 N|          455|
+---------+---------+-------------------+-------------------+---------------+-----------------+-----------------+------------------+------------------+------------------+-------------+
only showing top 1 row



In [9]:
def h_distance(lng1, lat1, lng2, lat2):
    # km
    # toRadians: Convert angles from degrees to radians
    lat1 = toRadians(lat1)
    lng1 = toRadians(lng1)
    lat2 = toRadians(lat2) 
    lng2 = toRadians(lng2)
    AVG_EARTH_RADIUS = 6371  #  km
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = sin(lat * 0.5) ** 2 +\
    cos(lat1) *\
    cos(lat2) *\
    sin(lng * 0.5) ** 2
    
    h = 2 * AVG_EARTH_RADIUS * asin(sqrt(d))
    return h

# Manhattan distance
def m_distance(lng1, lat1, lng2, lat2):
    # km 
    a = h_distance(lat1, lng1, lat1, lng2)
    b = h_distance(lat1, lng1, lat2, lng1)
    return a + b

# Convert time into hour-based type
def get_time(x):
    hour = udf(lambda s: datetime.strptime(s, '%Y-%m-%d %H:%M:%S').hour, IntegerType())
    minute = udf(lambda s: datetime.strptime(s, '%Y-%m-%d %H:%M:%S').minute, IntegerType())
    second = udf(lambda s: datetime.strptime(s, '%Y-%m-%d %H:%M:%S').second, IntegerType())
    
    h = hour(x)
    m = minute(x)
    s = second(x)
    
    time = h + m/60. + s/60./60.
    
    return time

# Calculate the direction from start point to destination
def get_direction(lng1, lat1, lng2, lat2):
    # theta
    AVG_EARTH_RADIUS = 6371  #  km
    lng_delta_rad = toRadians(lng2 - lng1)
    
    lat1 = toRadians(lat1)
    lng1 = toRadians(lng1)
    lat2 = toRadians(lat2) 
    lng2 = toRadians(lng2)
    
    y = sin(lng_delta_rad) * cos(lat2)
    x = cos(lat1) * sin(lat2) - sin(lat1) * cos(lat2) * cos(lng_delta_rad)
    return toDegrees(atan2(y, x))


parse_time_year = udf(lambda s: datetime.strptime(s, '%Y-%m-%d %H:%M:%S').year, IntegerType())
parse_time_month = udf(lambda s: datetime.strptime(s, '%Y-%m-%d %H:%M:%S').month, IntegerType())
parse_time_date = udf(lambda s: datetime.strptime(s, '%Y-%m-%d %H:%M:%S').day, IntegerType())
parse_time_weekday = udf(lambda s: datetime.strptime(s, '%Y-%m-%d %H:%M:%S').weekday(), IntegerType())
parse_time_hour = udf(lambda s: datetime.strptime(s, '%Y-%m-%d %H:%M:%S').hour, IntegerType())

In [10]:
# df. is a column / df.select is a dataframe / df.select.rdd is rdd
df = df.withColumn('pickup_year', parse_time_year(df.pickup_datetime))
df = df.withColumn('pickup_month', parse_time_month(df.pickup_datetime))
df = df.withColumn('pickup_date', parse_time_date(df.pickup_datetime))
df = df.withColumn('pickup_weekday', parse_time_weekday(df.pickup_datetime))
df = df.withColumn('pickup_hour', parse_time_hour(df.pickup_datetime))

In [11]:
df = df.withColumn('dropoff_year', parse_time_year(df.dropoff_datetime))
df = df.withColumn('dropoff_month', parse_time_month(df.dropoff_datetime))
df = df.withColumn('dropoff_date', parse_time_date(df.dropoff_datetime))
df = df.withColumn('dropoff_weekday', parse_time_weekday(df.dropoff_datetime))
df = df.withColumn('dropoff_hour', parse_time_hour(df.dropoff_datetime))

In [12]:
df = df.withColumn('H_distance', h_distance(df.pickup_longitude, df.pickup_latitude, df.dropoff_longitude, df.dropoff_latitude))

In [13]:
df = df.withColumn('pickup_time(hour)', get_time(df.pickup_datetime))
df = df.withColumn('dropoff_time(hour)', get_time(df.dropoff_datetime))

In [14]:
df = df.withColumn('m_distance', m_distance(df.pickup_longitude, df.pickup_latitude, df.dropoff_longitude, df.dropoff_latitude))

In [15]:
df = df.withColumn('direction', get_direction(df.pickup_longitude, df.pickup_latitude, df.dropoff_longitude, df.dropoff_latitude))

In [16]:
from pyspark.ml.feature import StringIndexer

In [17]:
# Convert category into numeric
indexer = StringIndexer(inputCol='store_and_fwd_flag', outputCol='store_and_fwd_flag_dummy').fit(df)

In [18]:
df = indexer.transform(df)

In [19]:
# Briefly look at what we have done here
df.select('dropoff_time(hour)', 'pickup_time(hour)', 'H_distance', 'm_distance', 'direction', 'store_and_fwd_flag_dummy').show(5)

+------------------+------------------+------------------+------------------+-------------------+------------------------+
|dropoff_time(hour)| pickup_time(hour)|        H_distance|        m_distance|          direction|store_and_fwd_flag_dummy|
+------------------+------------------+------------------+------------------+-------------------+------------------------+
|17.541666666666668|17.415277777777778|1.4985207796469109|2.0202916912391164|  99.97019564714783|                     0.0|
|0.9105555555555556|0.7263888888888889|1.8055071687958897|2.3474699407065938|-117.15376821269284|                     0.0|
|             12.18|             11.59| 6.385098495253531| 4.577755595731812|-159.68016515404983|                     0.0|
| 19.66111111111111|19.541944444444447|1.4854984227708028|0.6536447279420972|-172.73769967366482|                     0.0|
|13.636111111111111|13.515277777777778|1.1885884593338851| 0.342564799686101|   179.473584610839|                     0.0|
+---------------

# Weather data

In [20]:
weather = sqlCtx.read.format('com.databricks.spark.csv').options(header=True).load('central_park_weather.csv')

In [21]:
weather.columns

['STATION',
 'STATION_NAME',
 'DATE',
 'PRCP',
 'SNWD',
 'SNOW',
 'TMAX',
 'TMIN',
 'AWND']

In [22]:
weather.select('DATE').show(3)

+--------+
|    DATE|
+--------+
|20090101|
|20090102|
|20090103|
+--------+
only showing top 3 rows



In [23]:
parse_date_year = udf(lambda s: datetime.strptime(s, '%Y%m%d').year, IntegerType())
parse_date_month = udf(lambda s: datetime.strptime(s, '%Y%m%d').month, IntegerType())
parse_date_day = udf(lambda s: datetime.strptime(s, '%Y%m%d').day, IntegerType())

In [24]:
weather = weather.withColumn('year', parse_date_year(weather.DATE))
weather = weather.withColumn('month', parse_date_month(weather.DATE))
weather = weather.withColumn('day', parse_date_day(weather.DATE))

In [25]:
weather.select('DATE', 'year', 'month', 'day').show(3)

+--------+----+-----+---+
|    DATE|year|month|day|
+--------+----+-----+---+
|20090101|2009|    1|  1|
|20090102|2009|    1|  2|
|20090103|2009|    1|  3|
+--------+----+-----+---+
only showing top 3 rows



In [26]:
# Since the taxi data only contains 2016
temp = weather.filter(weather.year == 2016)

In [27]:
temp = temp.drop('STATION', 'STATION_NAME', 'DATE')

In [28]:
# Merge two dataframe based on same year, month, day
dff = df.join(temp, (df.pickup_year == temp.year) & (df.pickup_month == temp.month) & (df.pickup_date == temp.day))

In [29]:
# Show what we've got here
dff.select('pickup_datetime','year','month','day', 'TMAX', 'TMIN',).show(5)

+-------------------+----+-----+---+----+----+
|    pickup_datetime|year|month|day|TMAX|TMIN|
+-------------------+----+-----+---+----+----+
|2016-03-14 17:24:55|2016|    3| 14|  51|  40|
|2016-06-12 00:43:35|2016|    6| 12|  83|  62|
|2016-01-19 11:35:24|2016|    1| 19|  28|  16|
|2016-04-06 19:32:31|2016|    4|  6|  48|  30|
|2016-03-26 13:30:55|2016|    3| 26|  55|  38|
+-------------------+----+-----+---+----+----+
only showing top 5 rows



In [30]:
# Simply show how many trips in each day
a = dff.groupby('pickup_weekday').agg({'vendor_id':'count'})
a.show()

+--------------+----------------+
|pickup_weekday|count(vendor_id)|
+--------------+----------------+
|             1|          202749|
|             6|          195366|
|             3|          218574|
|             5|          220868|
|             4|          223533|
|             2|          210136|
|             0|          187418|
+--------------+----------------+



In [ ]:
sc.stop()